In [1]:
from __future__ import division
from __future__ import print_function  
import warnings
import pandas as pd
import gsp_support as gsp
import matplotlib.pyplot as plt
import numpy as np
pd.options.plotting.backend = "plotly"
import datetime

In [2]:
###########################################################################################################################

In [2]:
# data handling

# get the final ac_list just to be sure

survey = pd.read_csv('FINAL- Appliance stock survey and metadata 20Nov19.xlsx - FINAL- Appliance stock survey a.csv')
ac_list = list(survey.loc[survey['q_409_ac_yn']==1]['home_no'])
ac_list = list(set(ac_list) - set(['BR17','BR20','BR42']))

# folder to read the stage 1 result dfs

input_folder = 'FINAL/Disaggregation/gsp_output+stage1_filter/'
output_folder = 'FINAL/Disaggregation/stage2_filter/'

# read the original smart meter data df to get the voltage values from

full = pd.read_csv('D:/CEEW/Smart Meter Analysis/Data Acquisition and Preprocessing/Cleaned data 02Mar21_corrected.csv')

full['x_Timestamp'] = pd.to_datetime(full['x_Timestamp'])
full = full.set_index('x_Timestamp')

In [3]:
# read the csv files and add voltage sginal to it - then save all the data in a dataframe #total with a meter column

total = pd.DataFrame()
mp = []
for m in ac_list:
    csv = pd.read_csv(input_folder+m+'_gsp.csv')
    csv.index = pd.to_datetime(csv['x_Timestamp'])
    
    g = full.loc[full['meter']==m]
    mask = g['z_Avg Voltage (Volt)'].index.isin(csv.index)
    c = g['z_Avg Voltage (Volt)'].loc[mask]
    csv['voltage'] = c
    total = total.append(csv)
#     print(csv)
    mp.append([m]*csv.shape[0])

me =  [item for sublist in mp for item in sublist]
total['meter'] = me

In [4]:
total = total.drop(['x_Timestamp'],axis=1)

In [5]:
# len(total['meter'].unique())
total

,0,Power,Current,Comp,Comp2,AC Power,AC on,voltage,meter
x_Timestamp,,,,,,,,,
2019-05-30 00:00:00,0.00,1780.0,10.13,0,0,0.0,0,182.25,MH45
2019-05-30 00:03:00,8.14,1780.0,10.12,1,0,0.0,0,181.75,MH45
2019-05-30 00:06:00,8.13,3200.0,18.26,1,0,0.0,0,178.80,MH45
2019-05-30 00:09:00,0.00,1780.0,10.13,0,0,0.0,0,181.65,MH45
2019-05-30 00:12:00,0.00,1780.0,10.34,0,0,0.0,0,176.80,MH45
...,...,...,...,...,...,...,...,...,...
2020-12-28 23:45:00,0.00,100.0,0.51,0,0,0.0,0,248.19,BR18
2020-12-28 23:48:00,0.00,100.0,0.50,0,0,0.0,0,247.87,BR18
2020-12-28 23:51:00,0.00,100.0,0.50,0,0,0.0,0,247.71,BR18


In [4]:
###########################################################################################################################

In [37]:
# make the df look beautiful !
total.columns = ['gsp','power','current','compressor','final_compressor','ac_power','ac_on','voltage','meter']
# force zero ac where total household consumption < 1000
cols = ['compressor','final_compressor','ac_power','ac_on']
# for c in cols:
# total.loc[total['power']<=1000, cols] = 0
total.loc[total['ac_power']<=200, cols] = 0


ValueError: Length mismatch: Expected axis has 13 elements, new values have 9 elements

In [ ]:
###########################################################################################################################

In [38]:
# Rough Cell
# Note - use parallel notebooks for stage 2 filter

total['diff'] = total['compressor']-total['final_compressor']

# change the valyes of total here
total['c'] = 0  #compressor
total['acp'] = 0 # acpower
total['acon'] = 0 # acon

In [39]:
total

,gsp,power,current,compressor,final_compressor,ac_power,ac_on,voltage,meter,diff,c,acp,acon
x_Timestamp,,,,,,,,,,,,,
2019-05-30 00:00:00,0.00,1780.0,10.13,0,0,0.0,0,182.25,MH45,0,0,0,0
2019-05-30 00:03:00,8.14,1780.0,10.12,0,0,0.0,0,181.75,MH45,0,0,0,0
2019-05-30 00:06:00,8.13,3200.0,18.26,0,0,0.0,0,178.80,MH45,0,0,0,0
2019-05-30 00:09:00,0.00,1780.0,10.13,0,0,0.0,0,181.65,MH45,0,0,0,0
2019-05-30 00:12:00,0.00,1780.0,10.34,0,0,0.0,0,176.80,MH45,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-28 23:45:00,0.00,100.0,0.51,0,0,0.0,0,248.19,BR18,0,0,0,0
2020-12-28 23:48:00,0.00,100.0,0.50,0,0,0.0,0,247.87,BR18,0,0,0,0
2020-12-28 23:51:00,0.00,100.0,0.50,0,0,0.0,0,247.71,BR18,0,0,0,0


In [41]:
# MASKS



# Check for one meter

# total1 = total.loc[total['meter']=='MH27']

# changing the output of gsp here
t = pd.DataFrame()

grouped = total.groupby('meter')
for name,group in grouped:
    print(name)
    sub = grouped.get_group(name)
#     print('hey reached here1')
    gsp_result = sub 
    
    start_date = '2019-03-01' # from may
    end_date = '2019-10-31'  # till september

    mask1 = (gsp_result.index >= start_date) & (gsp_result.index <= end_date)
    start_date1 = '2020-03-01' # from may
    end_date1 = '2020-10-31'  # till september

    mask2 = (gsp_result.index >= start_date1) & (gsp_result.index < end_date1)
    mask3 = gsp_result['gsp']==0
    mask4 = gsp_result['ac_on']==0
    
    ac_ = gsp_result.loc[gsp_result['ac_on']==1]
    ac_not = gsp_result.loc[gsp_result['ac_on']==0]
    ac_on_overall_power = ac_['ac_power'].mean()
    base = ac_not['power'].mean()
    print(ac_on_overall_power)

    mask5 = (gsp_result['power']-ac_on_overall_power)>(base+200)
    
    mask = (mask1 | mask2) & mask3 & mask4 & mask5
    
    
#     print(gsp_result.loc[mask])
    gsp_result.loc[mask,'c'] = 1
    
    gsp_result.loc[mask,'acp'] = gsp_result['power']-base
    print('hey reached here final')
    ma = gsp_result['acp']>(ac_on_overall_power+1000)
    gsp_result.loc[ma,'acp'] = ac_on_overall_power
    
    t = t.append(gsp_result)
    
  


#     #     print(gsp_result['Power'][t['Ton']:t['Toff']].mean())
#         s = gsp_result['Power'][t['Ton']:t['Toff']].mean()-avg
#         gsp_result['AC Power'][t['Ton']:t['Toff']] = s
#     gsp_result['AC Power'][gsp_result['AC Power'] < 0] = 0            


BR06
1065.2912151259602
hey reached here final
BR07


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



984.813108149791
hey reached here final
BR08
954.6525729716077
hey reached here final
BR11
1310.21163773667
hey reached here final
BR12
1260.9333938511745
hey reached here final
BR13
1130.5651819297564
hey reached here final
BR18
1072.4780693651257
hey reached here final
BR22
948.7880242936933
hey reached here final
BR23
1085.257703468183
hey reached here final
BR24
1252.4810763425414
hey reached here final
BR31
1380.428382132868
hey reached here final
BR35
938.8504941709538
hey reached here final
BR36
705.7622767763584
hey reached here final
BR38
1255.0824318070127
hey reached here final
BR45
1140.4640684971748
hey reached here final
BR49
951.1485293814421
hey reached here final
BR51
1380.6384112563114
hey reached here final
MH02
1471.6748246221669
hey reached here final
MH07
1321.465736768425
hey reached here final
MH10
1381.4255124442186
hey reached here final
MH14
1124.3297532912982
hey reached here final
MH18
1202.704188798987
hey reached here final
MH21
918.7672828914073
hey reac

In [57]:
t.to_csv(output_folder+'before_stage2.csv')

In [44]:
len(t['meter'].unique())

31

In [46]:
# inspection of filtering


sample = t.loc[t['meter']=='MH27']

In [47]:
start_date = '2019-05-22' # from may
end_date = '2019-05-23'  # till september
mask = (sample.index >= start_date) & (sample.index < end_date)
# sample = e.loc[mask] 
g = sample.loc[mask] 

In [48]:
g['power'].plot()

In [56]:
g['final_compressor'].plot()

In [40]:
# mask = (mask1 | mask2) & mask3 & mask4 & mask5
g['ac_power'].plot()   

In [31]:
gsp_result.loc[mask]

,gsp,power,current,compressor,final_compressor,ac_power,ac_on,voltage,meter,diff,c,acp,acon
x_Timestamp,,,,,,,,,,,,,
2019-05-22 03:06:00,0.0,2580.0,10.83,0,0,0.000000,0,246.28,MH27,0,1,2173.785039,0
2019-05-22 03:09:00,0.0,2600.0,10.95,0,0,0.000000,0,248.52,MH27,0,1,2193.785039,0
2019-05-22 03:12:00,0.0,2540.0,10.77,0,0,0.000000,0,246.58,MH27,0,1,2133.785039,0
2019-05-22 03:15:00,0.0,2580.0,10.84,0,0,0.000000,0,247.44,MH27,0,1,2173.785039,0
2019-05-22 03:18:00,0.0,2680.0,11.08,0,0,0.000000,0,247.30,MH27,0,1,2273.785039,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-24 05:54:00,0.0,2500.0,10.63,0,0,0.000000,0,237.03,MH27,0,1,2093.785039,0
2020-09-24 07:57:00,0.0,2760.0,12.41,0,0,3109.653345,0,229.23,MH27,0,1,2353.785039,0
2020-10-15 19:51:00,0.0,2520.0,10.95,0,0,3848.320012,0,233.75,MH27,0,1,2113.785039,0
